# Import modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
import pytz
import datetime 
import astral
import astropy.time as aptime
import os
import glob
from scipy import stats

In [ ]:
from matplotlib import rc
from matplotlib import rcParams
rc('xtick', labelsize=15) 
rc('ytick', labelsize=15)

rc('axes', labelsize=20) 
rc('axes', linewidth=2)

rcParams['axes.titlesize'] = 15
rcParams['legend.fontsize'] = 15
rcParams['patch.linewidth'] = 1
#rcParams['axes.titlepad'] = 30
rcParams['axes.labelpad'] = 10

# file paths & save paths

In [ ]:
data_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/datatables/'

In [ ]:
save_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Results/sp_psd_slope_correlations'

In [ ]:
txt_file1 = pd.read_csv(data_path + 'IFS_RawContrast_Gemini_env_20180530_all_seeing.csv')
txt_file1_copy = copy.copy(txt_file1)
raw_contrast_data= pd.DataFrame(txt_file1_copy)
print len(raw_contrast_data)

raw_contrast_data['dts'] = aptime.Time(raw_contrast_data['MJDOBS'], format='mjd').datetime 
raw_contrast_data['dts'] = raw_contrast_data['dts'].dt.tz_localize(pytz.UTC)

# Read all the CSV files and concatentae all of them

In [ ]:
allFiles = glob.glob(data_path + "/sp_psd_summary*.csv")

list_ = []

for file_ in allFiles[0:3]:
    df = pd.read_csv(file_,index_col=0)
    list_.append(df)

frame = pd.concat(list_, axis = 0, ignore_index = True)

In [ ]:
txt_file2 = pd.read_csv(data_path + 'sp_psd_summary20181204.csv',index_col=0) 
txt_file2_copy = copy.copy(txt_file2)

sp_psd_data = pd.DataFrame(txt_file2_copy)
sp_psd_data['dts'] = pd.to_datetime(sp_psd_data.loc[:,'filename'].str.split('_').str.get(2) + '_' + sp_psd_data.loc[:,'filename'].str.split('_').str.get(3),
              utc=True, format = '%Y.%m.%d_%H.%M.%S')

sp_psd_data['dts'] = sp_psd_data['dts'].dt.tz_localize(pytz.UTC)

In [ ]:
sp_psd_data

# Filter data

In [ ]:
def filter_data(data, imag = 10., tau = 1., see = 2., all_cond = True):
    "This selects all rows of input dataframe that satisfy conditions set by the user"
    
    date1 = datetime.date(year = 2014, month = 1, day =1)
    
    #  AO system requirements
    cond1 = data['dts'] > date1 
    cond2 = data['COADDS'] == 1
    cond3 = (data['OBSMODE'] == 'H_coron')|(data['OBSMODE'] == 'Spec')
    cond4 = data['AOFRAMES'] == 1000

    #  Good seeing conditions
    cond5 = data['IMAG'] < imag #Bright Stars
    cond6 = data['MASSTAU'] > tau  # slow moving turbulence [ms]
    cond7 = data['DIMMSEE'] < see  # smaller scale turbulence ["]
    
    if all_cond:
        ind = np.where(cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7)[0]  
        print len(ind)
    else:
        ind = np.where(cond1 & cond2 & cond3 & cond4 & cond5)[0]  
        print len(ind)

    filtered_data = data.iloc[ind]
    #filtered_data = filtered_data.dropna(subset = ['CONTR040','cal_wfe','M1_avg','TAMBIENT','Outside_OE_temperature'])
    
    new_ind = np.arange(len(filtered_data))
    filtered_data = filtered_data.set_index(new_ind)
        
    return filtered_data

In [ ]:
r = filter_data(raw_contrast_data,all_cond=True)

# Merge PSD data with IFS data

In [ ]:
sp_psd_raw_IFS_data =  pd.merge_asof(sp_psd_data[['slope','dts','filename','path']].sort_values('dts'),raw_contrast_data, on='dts',tolerance=pd.Timedelta('1min'))

In [ ]:
sp_psd_raw_IFS_data = sp_psd_raw_IFS_data.dropna(subset=['slope','M1_avg'])

In [ ]:
print sum(np.isfinite(sp_psd_raw_IFS_data['TAMBIENT']))

# Plotting

No grouping

In [ ]:
%matplotlib notebook

plt.figure(figsize=[8,6])
x = (sp_psd_raw_IFS_data['M1_avg']-sp_psd_raw_IFS_data['TAMBIENT'])
y = sp_psd_raw_IFS_data['slope']

print(sum(np.isfinite(y)))
plt.scatter(x,y,alpha=.5,c='navy')

plt.ylabel('log of Spatial PSD Slope',fontsize=15)
plt.xlabel('Primary - Outside air [C]',fontsize=15)

#plt.savefig(save_path+'sp_psd_slope_vs_delT_20181205.png')
#plt.plot(sp_psd_raw_IFS_data['slope'],'.',alpha=.5)

Number of unique objects and nights

In [ ]:
print(len(pd.unique(sp_psd_raw_IFS_data['OBJNAME'])))
print(len(pd.unique(sp_psd_raw_IFS_data['night_number'])))

In [ ]:
sp_psd_raw_IFS_data.keys()

With grouping

In [ ]:
grouped = sp_psd_raw_IFS_data.groupby(['night_number', 'OBJNAME'], as_index= False).mean().dropna(subset = ['slope','M1_avg'])

In [ ]:
less = np.where(np.abs(grouped['M1_avg']-grouped['T_twr'])<1.)[0]
more = np.where(np.abs(grouped['M1_avg']-grouped['T_twr'])>=1.)[0]

grouped.loc[more,'slope'].mean()

In [ ]:
%matplotlib notebook

x = grouped['M1_avg']-grouped['TAMBIENT']
y = grouped['slope']
print(sum(np.isfinite(y)))


plt.figure(figsize=[8,6])
plt.scatter(x,y,c='red',marker='s')
plt.ylabel('log of Spatial PSD Slope',fontsize=15)
plt.xlabel('Primary - Outside air [C]',fontsize=15)
#plt.colorbar()


#plt.savefig(save_path+'sp_psd_slope_vs_delT_grouped_20181205.pdf')


#plt.savefig(save_path+'sp_psd_slope_vs_delT_night_20181201.pdf')
#plt.plot(sp_psd_raw_IFS_data['slope'],'.',alpha=.5)

In [ ]:
ind = np.where(grouped['slope'] == np.max(grouped['slope']))[0]

In [ ]:
def search_for_dome_seeing(data, sample_size, presence = True):
    
    cut = 0
    sample = set()
    
    if presence:
        
        while len(sample) <= sample_size:
            tau0 = data.MASSTAU.sort_values(ascending = False)[0:cut]
            mirror_to_air_temp = np.abs(data.M1_avg - data.T_twr).sort_values(ascending = False)[0:cut]
            contrast = data.CONTR040.sort_values(ascending = False)[0:cut]
            psd_slope = data.slope.sort_values(ascending = False)[0:cut]
            sample = set(tau0.index) & set(mirror_to_air_temp.index) & set(contrast.index) & set(psd_slope.index)
            cut = cut + 1
            
    else:
        
        while len(sample) <= sample_size:
            tau0 = data.MASSTAU.sort_values(ascending = False)[0:cut]
            mirror_to_air_temp = np.abs(data.M1_avg - data.T_twr).sort_values()[0:cut]
            contrast = data.CONTR040.sort_values()[0:cut]
            psd_slope = data.slope.sort_values(ascending = True)[0:cut]
            sample = set(tau0.index) & set(mirror_to_air_temp.index) & set(contrast.index) & set(psd_slope.index)
            cut = cut + 1
    return list(sample)  

In [ ]:
sp_psd_raw_IFS_data.loc[search_for_dome_seeing(sp_psd_raw_IFS_data,10,presence=True),
                       ['dts','CONTR040','M1_avg','TAMBIENT','MASSTAU','slope','IMAG','filename']].sort_values("CONTR040",ascending = True)

In [ ]:
sp_psd_raw_IFS_data.loc[search_for_dome_seeing(sp_psd_raw_IFS_data,10,presence=False),
                       ['dts','CONTR040','M1_avg','TAMBIENT','MASSTAU','slope','filename']].sort_values("CONTR040",ascending = False)